# AutoML Forecasting with Azure ML


<img src='https://github.com/retkowsky/images/blob/master/AzureMLservicebanniere.png?raw=true'>

This python notebook uses Azure ML service and its AUTOML features to automatically generate a time series model.
We are using a time series dataset with daily observations.
> https://docs.microsoft.com/en-us/python/api/overview/azure/ml/intro?view=azure-ml-py

In [ ]:
# Version Python 3.6 du notebook
import sys
print("Version Python:",sys.version)

In [ ]:
import datetime
print("Date :" , datetime.datetime.now())

## 1. Paramétrage


In [ ]:
import azureml.core
import pandas as pd
import numpy as np
import logging
import warnings

from pandas.tseries.frequencies import to_offset

# Squash warning messages for cleaner output in the notebook
warnings.showwarning = lambda *args, **kwargs: None

from azureml.core.workspace import Workspace
from azureml.core.experiment import Experiment
from azureml.train.automl import AutoMLConfig
from matplotlib import pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error

As part of the setup you have already created a <b>Workspace</b>. To run AutoML, you also need to create an <b>Experiment</b>. An Experiment corresponds to a prediction problem you are trying to solve, while a Run corresponds to a specific approach to the problem.

In [ ]:
import azureml.core
print("Version Azure ML service :", azureml.core.VERSION)

In [ ]:
ws = Workspace.from_config()

# choose a name for the run history container in the workspace
experiment_name = 'Exemple4-automlforecast'
# project folder
project_folder = './sample_projects/workshop4'

experiment = Experiment(ws, experiment_name)

output = {}
output['SDK version'] = azureml.core.VERSION
output['Workspace'] = ws.name
output['Resource Group'] = ws.resource_group
output['Location'] = ws.location
output['Project Directory'] = project_folder
output['Run History Name'] = experiment_name
pd.set_option('display.max_colwidth', -1)
outputDf = pd.DataFrame(data = output, index = [''])
outputDf.T

## 2. Reading data

In [ ]:
mydateparser = lambda x: pd.datetime.strptime(x, "%d/%m/%Y")
data = pd.read_csv("Pollution.csv" , encoding = "iso-8859-1", parse_dates = ['Date'] , 
date_parser = mydateparser)

In [ ]:
data

In [ ]:
data.columns

In [ ]:
data.dtypes

## 3. Descriptive statistics

In [ ]:
print("Lignes, colonnes :",data.shape)

## Statistiques descriptives

In [ ]:
data.describe()

## Matrice de corrélations

In [ ]:
data.corr()

## Graphiques

In [ ]:
data[['CO2']].plot(figsize=(20,8), linewidth=2, fontsize=10)
plt.title('CO2')
plt.xlabel('Date', fontsize=20);

In [ ]:
data[['Temperature']].plot(figsize=(20,8), linewidth=2, fontsize=10)
plt.title('Température')
plt.xlabel('Date', fontsize=20);

In [ ]:
data[['O3_Ozone']].plot(figsize=(20,8), linewidth=2, fontsize=10)
plt.title('Ozone')
plt.xlabel('Date', fontsize=20);

In [ ]:
data[['Vent']].plot(figsize=(20,8), linewidth=2, fontsize=10)
plt.title('Vent')
plt.xlabel('Date', fontsize=20);

In [ ]:
data[['Humidite']].plot(figsize=(20,8), linewidth=2, fontsize=10)
plt.title('Humidité')
plt.xlabel('Date', fontsize=20);

In [ ]:
data[['SO2_Dioxyde_de_soufre']].plot(figsize=(20,8), linewidth=2, fontsize=10)
plt.title('Dioxyde de soufre')
plt.xlabel('Date', fontsize=20);

In [ ]:
data[['Direction_vent']].plot(figsize=(20,8), linewidth=2, fontsize=10)
plt.title('Direction du vent')
plt.xlabel('Date', fontsize=20);

## 4. AutoML Forecasting
We are going to predict CO2

In [ ]:
target_column_name = 'CO2' #Target variable to predict
time_column_name = 'Date'  #Date variable
grain_column_names = []    #Optional. Needed for multiple time series ie location...

<img src="https://azurecomcdn.azureedge.net/mediahandler/acomblog/media/Default/blog/616f2d9f-eb6a-4f3b-af56-cc82197db001.png" height="50" width="600">

## 4.1 Split the data

The first split we make is into train and test sets. Note we are splitting on time.

In [ ]:
train = data[data[time_column_name] < '2019-10-01']
test = data[data[time_column_name] >= '2019-10-01']

X_train = train.copy()
y_train = X_train.pop(target_column_name).values

X_test = test.copy()
y_test = X_test.pop(target_column_name).values

print("Training dimensions:")
print(X_train.shape)
print(y_train.shape)
print()
print("Test dimensions:")
print(X_test.shape)
print(y_test.shape)

## 4.2 Setting forecast maximum horizon 

The forecast horizon is the number of periods into the future that the model should predict.

In [ ]:
max_horizon = 30

## 4.3 Azure AutoML specification

Instantiate a AutoMLConfig object. This defines the settings and data used to run the experiment.

|Property|Description|
|-|-|
|**task**|forecasting|
|**primary_metric**|This is the metric that you want to optimize.<br> Forecasting supports the following primary metrics <br><i>spearman_correlation</i><br><i>normalized_root_mean_squared_error</i><br><i>r2_score</i><br><i>normalized_mean_absolute_error</i>
|**iterations**|Number of iterations. In each iteration, Auto ML trains a specific pipeline on the given data|
|**iteration_timeout_minutes**|Time limit in minutes for each iteration.|
|**X**|(sparse) array-like, shape = [n_samples, n_features]|
|**y**|(sparse) array-like, shape = [n_samples, ], targets values.|
|**n_cross_validations**|Number of cross validation splits.|
|**country_or_region**|The country/region used to generate holiday features. These should be ISO 3166 two-letter country/region codes (i.e. 'US', 'GB').|
|**path**|Relative path to the project folder.  AutoML stores configuration files for the experiment under this folder. You can specify a new empty folder. 

> Documentation : https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-configure-auto-train

In [ ]:
automl_settings = {
    'time_column_name': time_column_name,
    'max_horizon': max_horizon,
    'target_lags': 10,
    
}

automl_config = AutoMLConfig(task='forecasting',
                             debug_log = 'automl4.log',
                             primary_metric='normalized_root_mean_squared_error',
                             iterations=5,
                             iteration_timeout_minutes = 5,
                             experiment_timeout_minutes = 15,
                             enable_early_stopping=True,
                             X=X_train,
                             y=y_train,
                             n_cross_validations=5,  
                             enable_voting_ensemble=False,
                             enable_stack_ensemble=False,
                             path=project_folder,
                             **automl_settings)

> Normalized root-mean-square deviation. This value is commonly referred to as the normalized root-mean-square deviation or error ( NRMSD or NRMSE ), and often expressed as a percentage, where lower values indicate less residual variance. https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-configure-auto-train

In [ ]:
%%time
local_run = experiment.submit(automl_config, show_output=True)

## Notebook Widget

In [ ]:
from azureml.widgets import RunDetails
RunDetails(local_run).show()

## Link to Azure Portal

In [ ]:
local_run

### 4.4 Retrieve the Best Model
Below we select the best pipeline from our iterations. The **get_output** method on automl_classifier returns the best run and the fitted model for the last fit invocation. There are overloads on get_output that allow you to retrieve the best run and fitted model for any logged metric or a particular iteration.

In [ ]:
best_run, fitted_model = local_run.get_output()
fitted_model.steps

In [ ]:
fitted_model.get_params

In [ ]:
fitted_model.steps

### 4.5 View the engineered names for featurized data

In [ ]:
fitted_model.named_steps['timeseriestransformer'].get_engineered_feature_names()

### 4.6 View the featurization summary

You can also see what featurization steps were performed on different raw features in the user data. For each raw feature in the user data, the following information is displayed:

- Raw feature name
- Number of engineered features formed out of this raw feature
- Type detected
- If feature was dropped
- List of feature transformations for the raw feature

In [ ]:
# Get the featurization summary as a list of JSON
featurization_summary = fitted_model.named_steps['timeseriestransformer'].get_featurization_summary()
# View the featurization summary as a pandas dataframe
pd.DataFrame.from_records(featurization_summary)

### 4.7 Evaluate

We now use the best fitted model from the AutoML Run to make forecasts for the **test set.**  

We always score on the original dataset whose schema matches the training set schema.

In [ ]:
X_test.head()

In [ ]:
def align_outputs(y_predicted, X_trans, X_test, y_test, predicted_column_name='predicted',
                  horizon_colname='horizon_origin'):
    """
    Demonstrates how to get the output aligned to the inputs
    using pandas indexes. Helps understand what happened if
    the output's shape differs from the input shape, or if
    the data got re-sorted by time and grain during forecasting.
    
    Typical causes of misalignment are:
    * we predicted some periods that were missing in actuals -> drop from eval
    * model was asked to predict past max_horizon -> increase max horizon
    * data at start of X_test was needed for lags -> provide previous periods
    """
    df_fcst = pd.DataFrame({predicted_column_name : y_predicted,
                            horizon_colname: X_trans[horizon_colname]})
    # y and X outputs are aligned by forecast() function contract
    df_fcst.index = X_trans.index
    
    # align original X_test to y_test    
    X_test_full = X_test.copy()
    X_test_full[target_column_name] = y_test

    # X_test_full's index does not include origin, so reset for merge
    df_fcst.reset_index(inplace=True)
    X_test_full = X_test_full.reset_index().drop(columns='index')
    together = df_fcst.merge(X_test_full, how='right')
    
    # drop rows where prediction or actuals are nan 
    # happens because of missing actuals 
    # or at edges of time due to lags/rolling windows
    clean = together[together[[target_column_name, predicted_column_name]].notnull().all(axis=1)]
    return(clean)

def do_rolling_forecast(fitted_model, X_test, y_test, max_horizon, freq='D'):
    """
    Produce forecasts on a rolling origin over the given test set.
    
    Each iteration makes a forecast for the next 'max_horizon' periods 
    with respect to the current origin, then advances the origin by the horizon time duration. 
    The prediction context for each forecast is set so that the forecaster uses 
    the actual target values prior to the current origin time for constructing lag features.
    
    This function returns a concatenated DataFrame of rolling forecasts.
     """
    df_list = []
    origin_time = X_test[time_column_name].min()
    while origin_time <= X_test[time_column_name].max():
        # Set the horizon time - end date of the forecast
        horizon_time = origin_time + max_horizon * to_offset(freq)
        
        # Extract test data from an expanding window up-to the horizon 
        expand_wind = (X_test[time_column_name] < horizon_time)
        X_test_expand = X_test[expand_wind]
        y_query_expand = np.zeros(len(X_test_expand)).astype(np.float)
        y_query_expand.fill(np.NaN)
        
        if origin_time != X_test[time_column_name].min():
            # Set the context by including actuals up-to the origin time
            test_context_expand_wind = (X_test[time_column_name] < origin_time)
            context_expand_wind = (X_test_expand[time_column_name] < origin_time)
            y_query_expand[context_expand_wind] = y_test[test_context_expand_wind]
        
        # Make a forecast out to the maximum horizon
        y_fcst, X_trans = fitted_model.forecast(X_test_expand, y_query_expand)
        
        # Align forecast with test set for dates within the current rolling window 
        trans_tindex = X_trans.index.get_level_values(time_column_name)
        trans_roll_wind = (trans_tindex >= origin_time) & (trans_tindex < horizon_time)
        test_roll_wind = expand_wind & (X_test[time_column_name] >= origin_time)
        df_list.append(align_outputs(y_fcst[trans_roll_wind], X_trans[trans_roll_wind],
                                     X_test[test_roll_wind], y_test[test_roll_wind]))
        
        # Advance the origin time
        origin_time = horizon_time
    
    return pd.concat(df_list, ignore_index=True)

# Let's do the forecast

In [ ]:
df_all = do_rolling_forecast(fitted_model, X_test, y_test, max_horizon)

In [ ]:
df_all.head(100)

> We now calculate some error metrics for the forecasts and vizualize the predictions vs. the actuals.

In [ ]:
def APE(actual, pred):
    """
    Calculate absolute percentage error.
    Returns a vector of APE values with same length as actual/pred.
    """
    return 100*np.abs((actual - pred)/actual)

def MAPE(actual, pred):
    """
    Calculate mean absolute percentage error.
    Remove NA and values where actual is close to zero
    """
    not_na = ~(np.isnan(actual) | np.isnan(pred))
    not_zero = ~np.isclose(actual, 0.0)
    actual_safe = actual[not_na & not_zero]
    pred_safe = pred[not_na & not_zero]
    return np.mean(APE(actual_safe, pred_safe))

In [ ]:
print("Simple forecasting model")
rmse = np.sqrt(mean_squared_error(df_all[target_column_name], df_all['predicted']))
print("[Test Data] \nRoot Mean squared error: %.3f" % rmse)
mae = mean_absolute_error(df_all[target_column_name], df_all['predicted'])
print('mean_absolute_error score: %.3f' % mae)
print('MAPE: %.3f' % MAPE(df_all[target_column_name], df_all['predicted']))

# Plot outputs
%matplotlib inline
plt.figure(figsize=(20, 10))
plt.title('Model Prediction vs Actual')
test_pred = plt.scatter(df_all[target_column_name], df_all['predicted'], color='green')
test_test = plt.scatter(y_test, y_test, color='blue')
plt.legend((test_pred, test_test), ('Model prediction', 'Actual'), loc='upper left', fontsize=12)
plt.show()

In [ ]:
df_all.groupby('horizon_origin').apply(
    lambda df: pd.Series({'MAPE': MAPE(df[target_column_name], df['predicted']),
                          'RMSE': np.sqrt(mean_squared_error(df[target_column_name], df['predicted'])),
                          'MAE': mean_absolute_error(df[target_column_name], df['predicted'])}))

In [ ]:
df_all_APE = df_all.assign(APE=APE(df_all[target_column_name], df_all['predicted']))
APEs = [df_all_APE[df_all['horizon_origin'] == h].APE.values for h in range(1, max_horizon + 1)]

%matplotlib inline
plt.figure(figsize=(20, 8))
plt.boxplot(APEs)
plt.yscale('log')
plt.xlabel('horizon')
plt.ylabel('APE (%)')
plt.title('Absolute Percentage Errors by Forecast Horizon')

plt.show()

In [ ]:
print("Simple forecasting model")
rmse = np.sqrt(mean_squared_error(df_all[target_column_name], df_all['predicted']))
print("[Test Data] \nRoot Mean squared error: %.3f" % rmse)
mae = mean_absolute_error(df_all[target_column_name], df_all['predicted'])
print('mean_absolute_error score: %.3f' % mae)
print('MAPE: %.3f' % MAPE(df_all[target_column_name], df_all['predicted']))

# Plot outputs
%matplotlib inline
plt.figure(figsize=(20, 8))
pred, = plt.plot(df_all[time_column_name], df_all['predicted'], color='green')
actual, = plt.plot(df_all[time_column_name], df_all[target_column_name], color='blue')
plt.xticks(fontsize=8)
plt.legend((pred, actual), ('Model Prediction', 'Actual'), loc='upper left', fontsize=12)
plt.title('Prediction vs. Actual Time-Series')

plt.show()

In [ ]:
# Prediction
df_all.head(100)

## 5. AutoML Forecasting with lags


<img src="https://azurecomcdn.azureedge.net/mediahandler/acomblog/media/Default/blog/80748202-8fef-4d74-b8b1-bdd27e6838aa.png">

### Lags parameter

<img src="https://azurecomcdn.azureedge.net/mediahandler/acomblog/media/Default/blog/6690a165-f602-4c2e-ba5d-24437505ef69.png">

### Target Rolling Windows Size parameter

<img src="https://azurecomcdn.azureedge.net/mediahandler/acomblog/media/Default/blog/d03d6941-a7b8-4023-a4ab-d8f56751bfe4.png">

In [ ]:
time_series_settings_with_lags = {
    'time_column_name': time_column_name,
    'max_horizon': max_horizon,
    'country_or_region': 'FR',
    'target_lags': 10,
    'target_rolling_window_size': 30
}

automl_config_lags = AutoMLConfig(task='forecasting',
                                  debug_log='automlwork42.log',
                                  primary_metric='normalized_root_mean_squared_error',
                                  iterations=3,
                                  iteration_timeout_minutes=10,
                                  enable_voting_ensemble=False,
                                  enable_stack_ensemble=False,
                                  X=X_train,
                                  y=y_train,
                                  n_cross_validations=3,
                                  path=project_folder,
                                  verbosity=logging.INFO,
                                  **time_series_settings_with_lags)

In [ ]:
%%time
local_run_lags = experiment.submit(automl_config_lags, show_output=True)

In [ ]:
local_run_lags

In [ ]:
from azureml.widgets import RunDetails
RunDetails(local_run_lags).show()

In [ ]:
children = list(local_run_lags.get_children())
metricslist = {}
for run in children:
    properties = run.get_properties()
    metrics = {k: v for k, v in run.get_metrics().items() if isinstance(v, float)}
    metricslist[int(properties['iteration'])] = metrics

rundata = pd.DataFrame(metricslist).sort_index(1)
rundata

In [ ]:
best_run, fitted_model = local_run_lags.get_output()
fitted_model.steps

In [ ]:
fitted_model.get_params

In [ ]:
fitted_model.steps

In [ ]:
fitted_model.named_steps['timeseriestransformer'].get_engineered_feature_names()

In [ ]:
df_all2 = do_rolling_forecast(fitted_model, X_test, y_test, max_horizon)

In [ ]:
print("Simple forecasting model")
rmse = np.sqrt(mean_squared_error(df_all2[target_column_name], df_all2['predicted']))
print("[Test Data] \nRoot Mean squared error: %.3f" % rmse)
mae = mean_absolute_error(df_all2[target_column_name], df_all2['predicted'])
print('mean_absolute_error score: %.3f' % mae)
print('MAPE: %.3f' % MAPE(df_all2[target_column_name], df_all2['predicted']))

# Plot outputs
%matplotlib inline
plt.figure(figsize=(20, 10))
plt.title('Absolute Percentage Errors by Forecast Horizon')
test_pred = plt.scatter(df_all2[target_column_name], df_all2['predicted'], color='green')
test_test = plt.scatter(y_test, y_test, color='blue')
plt.legend((test_pred, test_test), ('Model Prediction', 'Actual'), loc='upper left', fontsize=12)
plt.show()

In [ ]:
df_all2.groupby('horizon_origin').apply(
    lambda df: pd.Series({'MAPE': MAPE(df[target_column_name], df['predicted']),
                          'RMSE': np.sqrt(mean_squared_error(df[target_column_name], df['predicted'])),
                          'MAE': mean_absolute_error(df[target_column_name], df['predicted'])}))


df_all2_APE = df_all2.assign(APE=APE(df_all2[target_column_name], df_all2['predicted']))
APEs = [df_all2_APE[df_all2['horizon_origin'] == h].APE.values for h in range(1, max_horizon + 1)]

In [ ]:
print("Simple forecasting model")
rmse = np.sqrt(mean_squared_error(df_all2[target_column_name], df_all2['predicted']))
print("[Test Data] \nRoot Mean squared error: %.3f" % rmse)
mae = mean_absolute_error(df_all2[target_column_name], df_all2['predicted'])
print('mean_absolute_error score: %.3f' % mae)
print('MAPE: %.3f' % MAPE(df_all2[target_column_name], df_all2['predicted']))

# Plot outputs
%matplotlib inline
plt.figure(figsize=(20, 8))
pred, = plt.plot(df_all2[time_column_name], df_all2['predicted'], color='green')
actual, = plt.plot(df_all2[time_column_name], df_all2[target_column_name], color='blue')
plt.xticks(fontsize=8)
plt.legend((pred, actual), ('Model Prediction', 'Actual'), loc='upper left', fontsize=12)
plt.title('Prediction vs. Actual Time-Series')

plt.show()

In [ ]:
df_all2_APE.tail(10)

> **Get started with time-series forecasting in automated ML.** With these new capabilities automated ML increases support more complex forecasting scenarios, provides more control to configure training data using lags and window aggregation and improves accuracy with new holiday featurization and ROCV. Azure Machine Learning aims to enable data scientists of all skill levels to use powerful machine learning technology that simplifies their processes and reduces the time spent training models. Get started by visiting our documentation and let us know what you think - we are committed to make automated ML better for you!

In [ ]:
best_run, fitted_model = local_run.get_output()
print(best_run)
print()
print(fitted_model)
print()
best_run_metrics = best_run.get_metrics()
for metric_name in best_run_metrics:
    metric = best_run_metrics[metric_name]
    print(metric_name, metric)

In [ ]:
from azureml.core import Model

best_run.register_model(model_path='outputs/model.pkl', model_name='Exemple4-AutoML-Forecast',
                        tags={'Training context':'Azure Auto ML'},
                        properties={'R2': best_run_metrics['r2_score'], 'RMSE': best_run_metrics['normalized_root_mean_squared_error']})

In [ ]:
# Liste des modèles référencés
for model in Model.list(ws):
    print(model.name, 'version =', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')

> Documentation : https://docs.microsoft.com/en-us/azure/machine-learning/service/concept-automated-ml<br>
https://docs.microsoft.com/en-us/azure/machine-learning/how-to-configure-auto-train

<img src="https://github.com/retkowsky/images/blob/master/Powered-by-MS-Azure-logo-v2.png?raw=true" height="300" width="300">